In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from spider2_utils import load_csv_database


-setup-

In [ ]:
import pandas as pd

_database = load_csv_database("delivery_center", rows_limit=-1)

orders = _database["orders"]
stores = _database["stores"]
hubs = _database["hubs"]

Step 1: # Merge orders → stores → hubs

In [ ]:
orders_merged = (
    orders
    .merge(stores[['store_id', 'hub_id']], on='store_id', how='left')
    .merge(hubs[['hub_id', 'hub_name']], on='hub_id', how='left')
)

Step 2: Filter for 'FINISHED' orders

In [ ]:
finished_orders = orders_merged[orders_merged['order_status'] == 'FINISHED']

Step 3: Get 'February' orders 

In [ ]:
february_orders = (
    finished_orders[finished_orders['order_created_month'] == 2]
    .groupby('hub_name', as_index=False)
    .agg(orders_february=('order_id', 'count'))
)

Step 4: Get 'March' orders

In [ ]:
march_orders = (
    finished_orders[finished_orders['order_created_month'] == 3]
    .groupby('hub_name', as_index=False)
    .agg(orders_march=('order_id', 'count'))
)

Step 5: Merge february orders with march orders on the hub_name

In [ ]:

merged = february_orders.merge(march_orders, on='hub_name', how='left')

Step 6: Drop rows with missing March data (only keep hubs that had orders in March too)

In [ ]:
filtered = merged[
    (merged['orders_february'] > 0) &
    (merged['orders_march'] > 0) &
    (((merged['orders_march'] - merged['orders_february']) / merged['orders_february']) > 0.2)
]

Step 7: Get the final result

In [ ]:
result = filtered[['hub_name']]

In [ ]:
result